In [23]:
# coding: utf-8
from __future__ import print_function,division,absolute_import

In [24]:
%matplotlib inline

In [26]:
import os
import sys
# sys.path.insert(0,os.path.realpath("../slim"))
# sys.path.insert(0,os.path.realpath(".."))

In [27]:
from matplotlib import pyplot as plt
from PIL import Image

In [28]:
import numpy as np
import tensorflow as tf

assert tf.__version__ >= '1.4.0', \
    ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

tag_constants = tf.saved_model.tag_constants

## model import

In [30]:
export_dir = 'export/saved_model' # 'export/Servo/1533281229'

In [31]:
saved_model_dir = export_dir
tag_set = tag_constants.SERVING

In [32]:
from tensorflow.python.tools import saved_model_utils

In [33]:
meta_graph_def = saved_model_utils.get_meta_graph_def(saved_model_dir,tag_set)
signature_def = meta_graph_def.signature_def['serving_default']

In [34]:
inputs_map = {k: input_.name for k,input_ in signature_def.inputs.items()}
inputs_map

{'inputs': 'image_tensor:0'}

In [35]:
outputs_map = {k: output_.name for k,output_ in signature_def.outputs.items()}
outputs_map

{'detection_boxes': 'detection_boxes:0',
 'detection_classes': 'detection_classes:0',
 'detection_scores': 'detection_scores:0',
 'num_detections': 'num_detections:0'}

## loading label map

In [36]:
PATH_TO_LABELS = './datasetname_label_map.pbtxt'
NUM_CLASSES = 40

In [37]:
# from object_detection.utils import label_map_util
import label_map_util

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=NUM_CLASSES,
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [38]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [39]:
import glob
# TEST_IMAGE_PATHS = glob.glob('images/*.jpg')
TEST_IMAGE_PATHS = sorted(glob.glob('Test_ImageSet_datasetname/Val_tf/*'))
#TEST_IMAGE_PATHS

In [40]:
import os

def image_cache_path(filepath):
    filepath_base, _ = os.path.splitext(filepath)
    dir_, base_ = os.path.split(filepath_base)
    dirbase_ = os.path.basename(dir_)
    cache_path = 'cache/' + dirbase_ + '__' + base_ + '.png'
    return cache_path
    
if not os.path.isdir('cache'):
    os.makedirs('cache')

In [41]:
# tensorflow setup

In [42]:
tf.reset_default_graph()
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess = tf.Session(config=config)
tf.saved_model.loader.load(sess, [tag_constants.SERVING], export_dir);
image_tensor_ = tf.get_default_graph().get_tensor_by_name(inputs_map['inputs'])
tensor_dict = {k: tf.get_default_graph().get_tensor_by_name(v) for k, v in outputs_map.items()}

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.


In [43]:
image_tensor_

<tf.Tensor 'image_tensor:0' shape=(?, ?, ?, 3) dtype=uint8>

In [44]:
# dry-run
image_path = TEST_IMAGE_PATHS[0]
image_np = load_image_into_numpy_array(Image.open(image_path))
image_np_expanded = np.expand_dims(image_np, axis=0)
output_dict = sess.run(tensor_dict,feed_dict={image_tensor_: image_np_expanded})

In [45]:
import time
import visualization_utils as vis_util


im_names = TEST_IMAGE_PATHS


t_start = time.time()
t_inference_elapsed_sum = 0.0
for image_path in im_names:
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    #image_np = load_image_into_numpy_array(Image.open(image_path))
    image_np = cv2.imread(image_path, flags=cv2.IMREAD_COLOR )
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Run inference
    t_inference_start = time.time()
    output_dict = sess.run(tensor_dict,feed_dict={image_tensor_: image_np_expanded})
    t_inference_elapsed = (time.time() - t_inference_start)
    t_inference_elapsed_sum += t_inference_elapsed
    
    # textual report
    detection_boxes = output_dict['detection_boxes'][0]
    detection_classes = output_dict['detection_classes'][0].astype(int)
    detection_scores = output_dict['detection_scores'][0]
    
    detection_masks = None
    if output_dict.get('detection_masks'):
        detection_masks, _ = output_dict.get('detection_masks')
    
    print('Image:',image_path,'t_inference_elapsed:',t_inference_elapsed)

    image_size = image_np.shape[:2] * 2

    true_boxes_index = np.where(detection_scores > 0.5)[0]
    for i, box in enumerate(detection_boxes[true_boxes_index]):
        print('  Box:',i,'class:',category_index[detection_classes[i]]['name'],
              'score:',detection_scores[i],'box:',(box * image_size).astype(int))

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        detection_boxes,
        detection_classes,
        detection_scores,
        category_index,
        instance_masks=detection_masks, # output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(image_cache_path(image_path))
    plt.close()
t_elapsed = time.time() - t_start
t_elapsed /= len(im_names)
t_inference_elapsed_sum /= len(im_names)
print(('mean t_elapsed',t_elapsed,'mean t_inference_elapsed',t_inference_elapsed_sum))

Image: Test_ImageSet_datasetname/Val_tf/1.png t_inference_elapsed: 0.2299199104309082
  Box: 0 class: turn_right score: 0.89095473 box: [ 395 1213  498 1321]
Image: Test_ImageSet_datasetname/Val_tf/10.png t_inference_elapsed: 0.22524666786193848
  Box: 0 class: go_straight score: 0.96747535 box: [ 73 169 156 250]
  Box: 1 class: turn_right score: 0.96560985 box: [ 256 1279  385 1403]
Image: Test_ImageSet_datasetname/Val_tf/11.png t_inference_elapsed: 0.2056283950805664
  Box: 0 class: turn_right_1to2 score: 0.9964594 box: [ 23 863 100 921]
Image: Test_ImageSet_datasetname/Val_tf/2.png t_inference_elapsed: 0.2189936637878418
  Box: 0 class: go_straight score: 0.74454 box: [121 236 189 305]
Image: Test_ImageSet_datasetname/Val_tf/20181023_095543.png t_inference_elapsed: 0.22465085983276367
Image: Test_ImageSet_datasetname/Val_tf/20181023_095623.png t_inference_elapsed: 0.2267765998840332
  Box: 0 class: turn_right score: 0.98333573 box: [ 290 1366  428 1499]
  Box: 1 class: go_straight s

Image: Test_ImageSet_datasetname/Val_tf/datasetname_122.png t_inference_elapsed: 0.21811914443969727
  Box: 0 class: right_urbanexpressway_entrance score: 0.99693066 box: [310 849 386 930]
Image: Test_ImageSet_datasetname/Val_tf/datasetname_1259.png t_inference_elapsed: 0.21162128448486328
  Box: 0 class: turn_right score: 0.9992925 box: [318 852 383 924]
  Box: 1 class: speedlimit_60 score: 0.99872273 box: [353  86 387 131]
Image: Test_ImageSet_datasetname/Val_tf/datasetname_166.png t_inference_elapsed: 0.21401143074035645
  Box: 0 class: right_urbanexpressway_entrance score: 0.9845813 box: [309 850 385 930]
  Box: 1 class: speedlimit_90 score: 0.8874212 box: [356  84 387 130]
Image: Test_ImageSet_datasetname/Val_tf/datasetname_2893.png t_inference_elapsed: 0.2162632942199707
Image: Test_ImageSet_datasetname/Val_tf/datasetname_2949.png t_inference_elapsed: 0.20908808708190918
  Box: 0 class: turn_left score: 0.9968076 box: [318 855 384 918]
  Box: 1 class: speedlimit_80 score: 0.85197

In [46]:
#debug

In [47]:
from traceback import print_exc
import os
import sys
import glob
import xml.etree.ElementTree as ET
from PIL import Image

def find_groundtruth(filepath):
    filepath_base, _ = os.path.splitext(filepath)
    dir_, base_ = os.path.split(filepath_base)
    try:
        gt_image_path = glob.glob('Test_ImageSet_datasetname/Val_tf/' + base_ + '.*')[0]
        gt_anno_path  = glob.glob('annotations/xmls/Val/' + base_ + '.xml')[0]
        return gt_image_path, gt_anno_path
    except:
        print(('gt not found for',filepath),file=sys.stderr)
        return None, None

class AnnotationNode:
    def update(self,**kwargs):
        vars(self).update(dict(**kwargs))
    def __repr__(self):
        return str(vars(self))

def parse_annotation(filename,image_dir=None,check_image=False):
    anno_ = ET.parse(filename).getroot()
    image_filename = anno_.findtext('image_filename')
    image_path = anno_.findtext('path')

    if image_dir is not None:
        dir_, file_ = os.path.split(image_path)
        image_path = os.path.join(image_dir,file_)

    if check_image:
        try:
            img = Image.open(image_path)
            del img
        except:
            print(('parse_annotation','image load failed',filename,image_path),file=sys.stderr)
            return None

    segmented = int(anno_.findtext('segmented'))
    objects = []
    for obj_ in anno_.findall('object'):
        obj_name = obj_.findtext('name')
        obj_pose = obj_.findtext('pose')
        obj_truncated = int(obj_.findtext('truncated'))
        obj_difficult = int(obj_.findtext('difficult'))
        obj_bndbox = [[
            int(obj_.find('bndbox').findtext('xmin')),
            int(obj_.find('bndbox').findtext('ymin')),
        ],[
            int(obj_.find('bndbox').findtext('xmax')),
            int(obj_.find('bndbox').findtext('ymax')),
        ]]
        obj = AnnotationNode()
        obj.update(
            name=obj_name,
            pose=obj_pose,
            truncated=obj_truncated,
            difficult=obj_difficult,
            bndbox=obj_bndbox,
        )
        objects.append(obj)
    anno = AnnotationNode()
    anno.update(
        anno_path=filename,
        image_path=image_path,
        segmented=segmented,
        objects=objects,
    )
    return anno

import matplotlib.pyplot as plt
from PIL import Image

def show_annotation(anno_,image_=None,ax=None):
    if image_ is None:
        image_ = anno_.image_path

    try:
        img = Image.open(image_)
    except:
        print_exc()
        print('*** Image read error:',image_,file=sys.stderr)
        return

    if ax is None:
        plt.figure()
        _, ax_ = plt.subplots(1,figsize=(11,11))
    else:
        ax_ = ax

    ax_.imshow(img)

    for obj in anno_.objects:
        min_, max_ = obj.bndbox
        minx, miny = min_
        maxx, maxy = max_
        ax_.add_patch(plt.Rectangle(min_,maxx-minx,maxy-miny,fill=False,color='r',lw=3))
        ax_.text(minx, miny - 15, obj.name,color='r',fontsize=12,bbox=dict(facecolor='blue',alpha=0.5))
    ax_.set_title('{:s}, {:d} objs'.format(os.path.basename(anno_.image_path),len(anno_.objects)))
    if ax is None:
        plt.tight_layout()
        plt.show();

In [49]:
from ipywidgets import interact,IntSlider
# from IPython.display import Image

@interact(num=IntSlider(0,0,len(im_names)-1,continuous_update=False))
def show_result(num):
    im_name = im_names[num]
    gt_image_path, gt_anno_path = find_groundtruth(im_name)
    if gt_image_path is not None:
        plt.figure(figsize=(11,6))
        ax = plt.subplot(121)
        result_image_path = image_cache_path(im_name)
        ax.imshow(Image.open(result_image_path))
        ax.set_title(result_image_path)
        ax.set_axis_off()
        ax = plt.subplot(122)
        anno = parse_annotation(gt_anno_path)
        show_annotation(anno, gt_image_path, ax=ax)
        ax.set_axis_off()
        plt.tight_layout()
        plt.show();


interactive(children=(IntSlider(value=0, continuous_update=False, description='num', max=78), Output()), _dom_…